In [1]:
import os
import ast

import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '1'
tqdm.pandas()

print(pd.__version__)
print(tf.__version__)

2023-07-03 14:21:10.059776: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-03 14:21:11.025899: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-03 14:21:11.097037: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-03 14:21:11.097631: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-03 14:21:12.266101: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

2.0.2
2.12.0


In [18]:
data_path = save_path = os.path.join('..', '..', 'data', 'mfcc_matix_data')
data_path

'../../data/mfcc_matix_data'

In [19]:
def conv_col_to_np_array(x):
    x = x[1:-1].split()
    x_l = list(map(np.float32, x))
    x_np = np.array(x_l)
    return x_np

df = pd.read_csv(os.path.join(data_path, 'aa.csv'), sep=',', index_col=False)
df['f_combined'] = df['f_combined'].apply(conv_col_to_np_array)
df.head()

,per_frame_idx,mb_name,phonon,asthma_status,f_combined
0,0,Gaurav,aa_0,Control,"[61.123413, 9.845393, 10.817043, 7.0560846, 7...."
1,1,Gaurav,aa_0,Control,"[26.805195, 23.643036, 19.462852, 15.425544, 1..."
2,2,Gaurav,aa_0,Control,"[30.08226, 25.409985, 19.316051, 13.504214, 9...."
3,3,Gaurav,aa_0,Control,"[27.535519, 24.26188, 20.139553, 16.368507, 13..."
4,4,Gaurav,aa_0,Control,"[19.49335, 17.590767, 15.298168, 13.29262, 11...."


In [58]:
# tf.convert_to_tensor(df['f_combined'].values[0], dtype=tf.float32)

<tf.Tensor: shape=(12,), dtype=float32, numpy=
array([61.123413 ,  9.845393 , 10.817043 ,  7.0560846,  7.272934 ,
        5.614048 ,  4.4911504,  4.1725154,  2.2860708,  2.8374383,
        2.339272 ,  2.7664695], dtype=float32)>

In [35]:
# df['f_combined'] = df['f_combined'].progress_apply(lambda x: tf.constant(x))
# Z = df['f_combined'].progress_apply(lambda x: tf.convert_to_tensor(x, dtype=tf.float32))

data_raw = np.zeros((df['f_combined'].shape[0], 12))

for i in range(df['f_combined'].shape[0]):
    data_raw[i] = df['f_combined'][i]

In [40]:
X = tf.convert_to_tensor(data_raw, dtype=tf.float32)

In [42]:
ds_tf = tf.data.Dataset.from_tensor_slices()

<tf.Tensor: shape=(12,), dtype=float32, numpy=
array([61.123413 ,  9.845393 , 10.817043 ,  7.0560846,  7.272934 ,
        5.614048 ,  4.4911504,  4.1725154,  2.2860708,  2.8374383,
        2.339272 ,  2.7664695], dtype=float32)>

In [5]:
df_np = df.to_numpy()
df_np.shape

(775426, 5)

In [6]:
X = df.loc[:, 'f_combined'].to_numpy()
X.shape

(775426,)

In [9]:
X_st = np.stack(X)
X_st.shape

(775426, 12)

In [7]:
type(df['f_combined'][0])

numpy.ndarray

In [11]:
ds_tf = tf.data.Dataset.from_tensor_slices((X_st, df['asthma_status'])).batch(16)

In [16]:
for x in ds_tf.take(4):
    print(x)

(<tf.Tensor: shape=(16, 12), dtype=float32, numpy=
array([[61.123413 ,  9.845393 , 10.817043 ,  7.0560846,  7.272934 ,
         5.614048 ,  4.4911504,  4.1725154,  2.2860708,  2.8374383,
         2.339272 ,  2.7664695],
       [26.805195 , 23.643036 , 19.462852 , 15.425544 , 12.429753 ,
        10.795815 , 10.223204 , 10.0231   ,  9.501163 ,  8.302703 ,
         6.56085  ,  4.79096  ],
       [30.08226  , 25.409985 , 19.316051 , 13.504214 ,  9.194826 ,
         6.7933583,  6.0150623,  6.2729626,  7.025842 ,  7.9010143,
         8.629334 ,  8.968523 ],
       [27.535519 , 24.26188  , 20.139553 , 16.368507 , 13.601135 ,
        11.807386 , 10.58957  ,  9.629306 ,  8.8983755,  8.510074 ,
         8.410082 ,  8.236045 ],
       [19.49335  , 17.590767 , 15.298168 , 13.29262  , 11.786129 ,
        10.563364 ,  9.328993 ,  8.018267 ,  6.815018 ,  5.9248962,
         5.365196 ,  4.9714656],
       [23.449383 , 19.190748 , 15.433406 , 13.775799 , 13.4842   ,
        12.870127 , 11.170859 ,  8.8

2023-07-03 12:05:01.776918: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype string and shape [775426]
	 [[{{node Placeholder/_1}}]]
